<a href="https://colab.research.google.com/github/AdrianDiazGaxiola/Proyecto-Final/blob/main/Ciclones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy
!pip install pandas

Se importan las librerías a utilizar

In [2]:
import pandas as pd
import numpy as np

Se lee y guarda en un dataframe los datos del archivo .csv con los datos de los ciclones que han impactado a México

In [3]:
url = "https://github.com/AdrianDiazGaxiola/Proyecto-Final/raw/main/Data/Ciclones.csv"

Ciclones = pd.read_csv(url, header=2, encoding='ISO-8859-1')

Se eliminan las filas y columnas que no se utilizarán

In [12]:
df = Ciclones

Cols = df.columns.tolist()

#Se eliminan las columnas que no se utilizarán (número, nombre, y categoría en impacto, ya que no se analizará la evolución de cada ciclón)
df = df.drop(columns = {Cols[0], Cols[1]})#, Cols[5]

#Se elimina el ciclón que aparece dos veces (solo se cosidera la ubicación del primer impacto)
df = df.drop(df.index[301])

Cols = df.columns.tolist()
print(Cols)
print(df)


['\nImpacto en tierra', '\nFecha de impacto', 'Categoría\nmáxima', 'Categoría \nen impacto', 'Velocidad \nen impacto (km/h)', '\nCosta']
                                   \nImpacto en tierra  ...    \nCosta
0                         Chamela - Pto Vallarta, Jal.  ...   Pacífico
1                                      Sta. Cruz, Nay.  ...   Pacífico
2           Villa de Tututepec de Melchor Ocampo, Oax.  ...   Pacífico
3                               Lázaro Cardenas, Mich.  ...   Pacífico
4                    30 Km Al Este De Tapachula, Chis.  ...   Pacífico
..                                                 ...  ...        ...
314  Inmediaciones de Chemuyil, Q. Roo y a 15 km al...  ...  Atlántico
315                                                NaN  ...        NaN
316                                                NaN  ...        NaN
317                                                NaN  ...        NaN
318                                                NaN  ...        NaN

[318 rows 

Se renombran los nombres de las columnas del dataframe

In [13]:
df.columns = ["Entidad_Impacto", "Fecha", "Categoría máxima", "Categoría en impacto", "Velocidad impacto", "Costa"]
print(df)

                                       Entidad_Impacto  ...      Costa
0                         Chamela - Pto Vallarta, Jal.  ...   Pacífico
1                                      Sta. Cruz, Nay.  ...   Pacífico
2           Villa de Tututepec de Melchor Ocampo, Oax.  ...   Pacífico
3                               Lázaro Cardenas, Mich.  ...   Pacífico
4                    30 Km Al Este De Tapachula, Chis.  ...   Pacífico
..                                                 ...  ...        ...
314  Inmediaciones de Chemuyil, Q. Roo y a 15 km al...  ...  Atlántico
315                                                NaN  ...        NaN
316                                                NaN  ...        NaN
317                                                NaN  ...        NaN
318                                                NaN  ...        NaN

[318 rows x 6 columns]


Se cambian los nombres de las ubicaciones a únicamente los nombres completos de los estados correspondiente

In [16]:
#Se cambia el tipo de dato de la columna con las entidades a tipo string, para poder utilizar las funciones a continuación
df['Entidad_Impacto'] = df['Entidad_Impacto'].astype(str) 

estados_abv = ['Ags.', 'Bc.', 'Bcs.', 'Camp.', 'Coah.', 'Col.', 'Chis.', 'Chih.', 'C.D.Mx.', 'Dgo.', 'Gto.', 'Gro.', 'Hgo.', 'Jal.', 'Edo. Méx.', 'Mich.', 'Mor.', 'Nay.', 'N.L.', 'Oax.','Pue.', 'Qro.', 'QRoo.', 'S.L.P.', 'Sin.', 'Son.', 'Tab.', 'Tamps.', 'Tlax.', 'Ver.', 'Yuc.','Zac.']

estados_comp = ['Aguascalientes', 'Baja California', 'Baja California Sur', 'Campeche', 'Coahuila', 'Colima', 'Chiapas', 'Chihuahua', 'Ciudad de Mexico', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 'Estado de Mexico', 'Michoacan', 'Morelos', 'Nayarit', 'Nuevo Leon', 'Oaxaca', 'Puebla', 'Queretaro', 'Quintana Roo', 'San Luis Potosi', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas', 'Tlaxcala', 'Veracruz', 'Yucatan', 'Zacatecas']

#En este ciclo, se busca si la ubicación contiene la abreviatura de algún estado, de ser así, se reemplaza el nombre de la ubicación por el nombre completo del estado correspondiente
j = 0
for i in estados_abv:
  df.loc[df['Entidad_Impacto'].str.contains(i), 'Entidad_Impacto'] = estados_comp[j]
  j += 1

#Se intercambian una por una las ubicaciones sin nombre de estado al estado correspondiente
df.loc[df['Entidad_Impacto'].str.contains('Tapachula'), 'Entidad_Impacto'] = 'Chiapas'
df.loc[df['Entidad_Impacto'].str.contains('Ciudad Obregón'), 'Entidad_Impacto'] = 'Sonora'
df.loc[df['Entidad_Impacto'].str.contains('Tuxpan'), 'Entidad_Impacto'] = 'Veracruz'
df.loc[df['Entidad_Impacto'].str.contains('Cabo San Lucas'), 'Entidad_Impacto'] = 'Baja California Sur'
df.loc[df['Entidad_Impacto'].str.contains('Los Cabos'), 'Entidad_Impacto'] = 'Baja California Sur'
df.loc[df['Entidad_Impacto'].str.contains('La Angostura'), 'Entidad_Impacto'] = 'Chiapas'
df.loc[df['Entidad_Impacto'].str.contains('San Lázaro'), 'Entidad_Impacto'] = 'Baja California Sur'
df.loc[df['Entidad_Impacto'].str.contains('Punta Abreojos'), 'Entidad_Impacto'] = 'Baja California Sur'
df.loc[df['Entidad_Impacto'].str.contains('San Juan de los Planes'), 'Entidad_Impacto'] = 'Baja California Sur'
df.loc[df['Entidad_Impacto'].str.contains('El Embudo'), 'Entidad_Impacto'] = 'Baja California Sur'
df.loc[df['Entidad_Impacto'].str.contains('Pico del Monte'), 'Entidad_Impacto'] = 'Guerrero'
df.loc[df['Entidad_Impacto'].str.contains('Cd. Hidalgo'), 'Entidad_Impacto'] = 'Chiapas'
df.loc[df['Entidad_Impacto'].str.contains('Laguna San Ignacio'), 'Entidad_Impacto'] = 'Baja California Sur'
df.loc[df['Entidad_Impacto'].str.contains('Pesquería'), 'Entidad_Impacto'] = 'Oaxaca'
df.loc[df['Entidad_Impacto'].str.contains('Santa Fé'), 'Entidad_Impacto'] = 'Quintana Roo'
df.loc[df['Entidad_Impacto'].str.contains('Tulum'), 'Entidad_Impacto'] = 'Quintana Roo'
df.loc[df['Entidad_Impacto'].str.contains('Chiapas'), 'Entidad_Impacto'] = 'Chiapas'
df.loc[df['Entidad_Impacto'].str.contains('Bahía Kino'), 'Entidad_Impacto'] = 'Sonora'
df.loc[df['Entidad_Impacto'].str.contains('Guaymas'), 'Entidad_Impacto'] = 'Sonora'
df.loc[df['Entidad_Impacto'].str.contains('Antón Lizardo'), 'Entidad_Impacto'] = 'Veracruz'
df.loc[df['Entidad_Impacto'].str.contains('Villa Guadalupe'), 'Entidad_Impacto'] = 'Sonora'
df.loc[df['Entidad_Impacto'].str.contains('Río Lagartos'), 'Entidad_Impacto'] = 'Yucatán'
df.loc[df['Entidad_Impacto'].str.contains('Ciudad Constitución'), 'Entidad_Impacto'] = 'Baja California Sur'
df.loc[df['Entidad_Impacto'].str.contains('San José de las Palomas'), 'Entidad_Impacto'] = 'Baja California'
df.loc[df['Entidad_Impacto'].str.contains('Playa Careyes'), 'Entidad_Impacto'] = 'Jalisco'
df.loc[df['Entidad_Impacto'].str.contains('Ciudad Constitución'), 'Entidad_Impacto'] = 'Baja California Sur'
df.loc[df['Entidad_Impacto'].str.contains('San José del Cabo'), 'Entidad_Impacto'] = 'Baja California Sur'
df.loc[df['Entidad_Impacto'].str.contains('Tenacatita'), 'Entidad_Impacto'] = 'Jalisco'
df.loc[df['Entidad_Impacto'].str.contains('Escuinapa'), 'Entidad_Impacto'] = 'Sinaloa'
df.loc[df['Entidad_Impacto'].str.contains('Isla San Ignacio'), 'Entidad_Impacto'] = 'Sinaloa'
df.loc[df['Entidad_Impacto'].str.contains('Lázaro Cárdenas'), 'Entidad_Impacto'] = 'Michoacán'
df.loc[df['Entidad_Impacto'].str.contains('Boca de Catán'), 'Entidad_Impacto'] = 'Tamaulipas'
df.loc[df['Entidad_Impacto'].str.contains('Armería'), 'Entidad_Impacto'] = 'Colima'
df.loc[df['Entidad_Impacto'].str.contains('La Pesca'), 'Entidad_Impacto'] = 'Tamaulipas'
df.loc[df['Entidad_Impacto'].str.contains('Lechuguillas'), 'Entidad_Impacto'] = 'Veracruz'
df.loc[df['Entidad_Impacto'].str.contains('Santa Rosalía'), 'Entidad_Impacto'] = 'Baja California Sur'
df.loc[df['Entidad_Impacto'].str.contains('Rancho Nuevo'), 'Entidad_Impacto'] = 'Tamaulipas'
df.loc[df['Entidad_Impacto'].str.contains('Chetumal'), 'Entidad_Impacto'] = 'Quintana Roo'
df.loc[df['Entidad_Impacto'].str.contains('San Diego'), 'Entidad_Impacto'] = 'Baja California'
df.loc[df['Entidad_Impacto'].str.contains('Ayutla'), 'Entidad_Impacto'] = 'Jalisco'

#Funciones utilizadas para identificar ubicaciones con nombres a reemplazar
entidades = df.Entidad_Impacto.tolist()
ent = np.array(entidades)
print(np.unique(ent))

print(df.head(5))


['Baja California' 'Baja California Sur' 'Campeche' 'Chiapas' 'Coahuila'
 'Colima' 'Guerrero' 'Jalisco' 'Michoacan' 'Morelos' 'Nayarit' 'Oaxaca'
 'Puebla' 'Queretaro' 'Quintana Roo' 'Sinaloa' 'Sonora' 'Tabasco'
 'Tamaulipas' 'Veracruz' 'Yucatan' 'nan']
  Entidad_Impacto       Fecha  ... Velocidad impacto     Costa
0         Jalisco  1983/05/27  ...              65.0  Pacífico
1         Nayarit  1983/05/28  ...              65.0  Pacífico
2          Oaxaca  1981/06/04  ...              30.0  Pacífico
3       Michoacan  1971/05/24  ...             140.0  Pacífico
4         Chiapas  2010/05/29  ...              75.0  Pacífico

[5 rows x 6 columns]
